In [5]:
import os #Operating System interactions
import tkinter as tk
from tkinter import messagebox
from catboost import CatBoostRegressor
import numpy as np
from PIL import Image, ImageTk, ImageEnhance

In [6]:
path = r'C:\Users\Sushant\OneDrive - Lamar University\Research\GGP\ML - Waste Glass Pozzolan\ML_Ensemble'
os.chdir(path)

<H1>Graphical User Interface</H1>

In [7]:
# ====== Load Pre-trained CatBoost Model ======
model = CatBoostRegressor()
try:
    model.load_model('catboost_model.cbm')
    print("Model loaded successfully")
except Exception as e:
    print("Could not load model:", e)


# ====== GUI App ======
root = tk.Tk()
root.title("Concrete Strength Prediction")
root.geometry("900x820")
root.configure(bg="white")


# ====== Top Image Loader (High Quality) ======
def load_header_image(base_name="GUI_Graphics"):
    """Load header image at best quality, resize proportionally, sharpen it."""
    for ext in (".png", ".jpg", ".jpeg"):
        try:
            img = Image.open(base_name + ext).convert("RGB")  # force RGB
            max_width = 780
            if img.width > max_width:  # scale proportionally
                ratio = max_width / img.width
                new_size = (max_width, int(img.height * ratio))
                img = img.resize(new_size, Image.LANCZOS)

            # Increase sharpness slightly
            img = ImageEnhance.Sharpness(img).enhance(1.3)

            return ImageTk.PhotoImage(img)
        except Exception:
            continue
    return None


# ====== Display Top Image ======
header_photo = load_header_image()
if header_photo:
    global global_header_photo
    global_header_photo = header_photo  # keep reference alive
    img_label = tk.Label(root, image=global_header_photo, bg="white")
    img_label.pack(pady=(15, 5))
else:
    fallback = tk.Label(
        root,
        text="(Header image not found: GUI_Graphics.png/.jpg/.jpeg)",
        font=("Arial", 10, "italic"),
        fg="#7B7D7D",
        bg="white"
    )
    fallback.pack(pady=(12, 3))


# ====== Parameters + Inputs ======
param_frame = tk.LabelFrame(
    root,
    text="⚙ Input Parameters ⚙",
    font=("Arial", 14, "bold"),
    fg="#1E8449",
    bg="white",
    padx=20,
    pady=20,
    relief="groove",
    bd=3
)
param_frame.pack(pady=10, anchor="center")

param_frame.grid_columnconfigure(0, weight=0)
param_frame.grid_columnconfigure(1, weight=1)


# Detect feature names (fallback to 11)
feature_names = model.feature_names_
if not feature_names:
    feature_names = [
        "GGP", "Replacement", "Cement",
        "Max size", "CA", "FA",
        "WC Ratio", "SiO2", "CaO", "Na2O", "Days"
    ]
print("Model expects", len(feature_names), "features:", feature_names)


# ====== Display Name Mapper ======
def display_name(raw: str) -> str:
    n = raw.lower().replace("_", " ").strip()

    # --- Chemistry FIRST so they don't get caught by 'ca' (coarse aggregate) ---
    if "sio2" in n:
        return "SiO₂ (%)"
    if "cao" in n:
        return "CaO (%)"
    if "na2o" in n:
        return "Na₂O (%)"

    # --- General materials/parameters ---
    if "ggp" in n or "glass" in n:
        return "Waste Glass Size (µm)"
    if "replacement" in n or "repl" in n:
        return "Cement Replacement (%)"
    if "cement" in n:
        return "Cement (kg/m³)"
    if "max" in n and "size" in n:
        return "Max Aggregate Size (mm)"
    if "wc" in n.replace(" ", "") or "w/c" in n:
        return "W/C Ratio (-)"
    if "day" in n or "curing" in n:
        return "Curing (Days)"

    # --- Aggregates (make CA rule exact-ish, not 'startswith("ca")') ---
    if n == "ca" or "coarse" in n:
        return "Coarse Aggregate (kg/m³)"
    if n == "fa" or "fine" in n:
        return "Fine Aggregate (kg/m³)"

    return raw



# ====== Build Inputs ======
entries = {}
for i, raw_text in enumerate(feature_names):
    text = display_name(raw_text)

    lbl = tk.Label(
        param_frame,
        text=f"X{i+1}: {text}",
        font=("Arial", 11),
        anchor="w",
        bg="white"
    )
    lbl.grid(row=i, column=0, sticky="w", padx=8, pady=5)

    entry = tk.Entry(
        param_frame,
        width=22,
        font=("Arial", 11),
        bg="white",
        relief="solid",
        bd=1,
        justify="center"
    )
    entry.grid(row=i, column=1, padx=8, pady=5, sticky="ew")
    entries[f"X{i+1}"] = entry


# ====== Prediction Logic ======
result_var = tk.StringVar()

def predict():
    try:
        input_data = []
        for i in range(1, len(feature_names) + 1):
            val_str = entries[f"X{i}"].get().strip()
            if val_str == "":
                raise ValueError(f"Missing value for X{i}")
            val = float(val_str)
            input_data.append(val)

        X_input = np.array(input_data).reshape(1, -1)
        print("➡ Input shape:", X_input.shape)

        prediction = model.predict(X_input)
        print("➡ Raw prediction:", prediction)

        result_var.set(f"CS = {prediction[0]:.4f} MPa")
    except Exception as e:
        messagebox.showerror("Error", f"Something went wrong:\n{e}")


def clear():
    for entry in entries.values():
        entry.delete(0, tk.END)
    result_var.set("")


# ====== Buttons ======
btn = tk.Button(
    param_frame,
    text="▶ Calculate",
    command=predict,
    font=("Arial", 12, "bold"),
    bg="#0B3B8C",
    fg="white",
    width=14,
    relief="raised",
    activebackground="#154360"
)
btn.grid(row=len(feature_names), column=0, pady=15, columnspan=2, sticky="ew")

clear_btn = tk.Button(
    param_frame,
    text="✖ Clear",
    command=clear,
    font=("Arial", 12, "bold"),
    bg="#BFC9CA",
    fg="black",
    width=14,
    relief="raised",
    activebackground="#A6ACAF"
)
clear_btn.grid(row=len(feature_names)+1, column=0, pady=5, columnspan=2, sticky="ew")


# ====== Result Box ======
result_frame = tk.Frame(root, bg="white", bd=2, relief="ridge")
result_frame.pack(pady=20)

result_label = tk.Label(
    result_frame,
    textvariable=result_var,
    font=("Arial", 20, "bold"),
    fg="red",
    bg="white",
    padx=20,
    pady=10
)
result_label.pack()

root.mainloop()


✅ Model loaded successfully
Model expects 11 features: ['GGP size', 'Replacement', 'Cement', 'Max size', 'CA', 'FA', 'WC Ratio', 'SiO2', 'CaO', 'Na2O', 'Days']
